In [109]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from networkx.classes.function import path_weight
from tqdm import tqdm


In [110]:

def create_graph(transports):
    graph = {}

    # Creating the graph with the list of transports
    for i,transport in tqdm(transports.iterrows()):
        id_emp_orig = transport['node_src']
        id_emp_dest = transport['node_dest']

        volume = transport['vol']

        # Graph is a dict (source) -> (destination, weight)
        if graph.get(id_emp_orig) is None:
            # if source is not in the graph we need to map it as source to the destination
            graph[id_emp_orig] = {id_emp_dest: {'weight': volume }}
        else:
            # if source is already in the graph
            # 1. new destination from that source: create the edge
            # 2. source already mapped to destination: increase the volume in that edge
            if graph[id_emp_orig].get(id_emp_dest) is None:
                graph[id_emp_orig][id_emp_dest] = {'weight': volume }
            else:
                graph[id_emp_orig][id_emp_dest]['weight'] += volume

    # In this context we want to maximize paths over the volume
    # Since most functions minimizes over the weight of the edges,
    # we need to invert the relation
    for source, targets in graph.items():
        for target, volume in targets.items():
            graph[source][target]['weight'] = -1 * volume['weight']

    return nx.DiGraph(graph)


In [111]:
# transportes_junho = pd.read_csv('../data/df_06.csv')
transportes_julho = pd.read_csv('../data/df_07.csv')
transportes_agosto = pd.read_csv('../data/df_08.csv')
transportes_setembro = pd.read_csv('../data/df_09.csv')
transportes_outubro= pd.read_csv('../data/df_10.csv')
transportes_novembro = pd.read_csv('../data/df_11.csv')
transportes_dezembro = pd.read_csv('../data/df_12.csv')

transporte_primeiro_semestre= pd.concat([transportes_julho, transportes_agosto, transportes_setembro, transportes_outubro, transportes_novembro, transportes_dezembro], ignore_index=True)


C:\Users\jaime\AppData\Local\Temp\ipykernel_16824\1596426700.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  transportes_julho = pd.read_csv('../data/df_07.csv')
C:\Users\jaime\AppData\Local\Temp\ipykernel_16824\1596426700.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  transportes_agosto = pd.read_csv('../data/df_08.csv')
C:\Users\jaime\AppData\Local\Temp\ipykernel_16824\1596426700.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  transportes_setembro = pd.read_csv('../data/df_09.csv')
C:\Users\jaime\AppData\Local\Temp\ipykernel_16824\1596426700.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  transportes_outubro= pd.read_csv('../data/df_10.csv')
C:\Users\jaime\AppData\Local\Temp\ipykernel_16824\1596426700.py:6: DtypeWarning: Columns (4) have mixed typ

In [112]:

transporte_primeiro_semestre= pd.concat([transportes_julho, transportes_agosto, transportes_setembro, transportes_outubro, transportes_novembro, transportes_dezembro], ignore_index=True)

rem_df = transporte_primeiro_semestre[['CPF_CNPJ_Rem', 'TpRem']].rename(columns={'CPF_CNPJ_Rem': 'CNPJ_CPF', 'TpRem': 'type'})
rem_df['source'] = 'Remetente'

# Criar DataFrame para CPF_CNPJ_Des e TpDes
des_df = transporte_primeiro_semestre[['CPF_CNPJ_Des', 'TpDes']].rename(columns={'CPF_CNPJ_Des': 'CNPJ_CPF', 'TpDes': 'type'})
des_df['source'] = 'Destinatário'

# Concatenar ambos os DataFrames
node_df= pd.concat([rem_df, des_df], ignore_index=True)
node_df= node_df.drop_duplicates('CNPJ_CPF')
node_df


,CNPJ_CPF,type,source
0,78952082000161,PTO_IBAMA,Remetente
1,9402361000139,PTO_IBAMA,Remetente
3,470994000100,PTO_IBAMA,Remetente
4,75218883000100,PTO_IBAMA,Remetente
5,5422729000170,PTO_IBAMA,Remetente
...,...,...,...
300472,28638514034,FINAL,Destinatário
300473,58371249004,FINAL,Destinatário
300474,50911406034,FINAL,Destinatário
300476,83227113015,FINAL,Destinatário


In [113]:

emp_type = {}

for i,node in node_df.iterrows():
  emp_type[node['CNPJ_CPF']] = node['type']


In [114]:
arestas = transporte_primeiro_semestre[['CPF_CNPJ_Rem', 'CPF_CNPJ_Des', 'Volume']].rename(columns={'CPF_CNPJ_Rem': 'node_src', 'CPF_CNPJ_Des': 'node_dest', 'Volume': 'vol'})

#  Removendo loops
arestas_1 = arestas[arestas['node_dest']!=arestas['node_src']]
arestas_1

,node_src,node_dest,vol
0,78952082000161,23342420987,0.4843
1,9402361000139,67972640904,0.2062
2,9402361000139,67972640904,0.4420
3,470994000100,369058950,0.8769
4,75218883000100,35649917,0.7840
...,...,...,...
150234,5148772000199,58371249004,0.3610
150235,91692392000152,50911406034,0.5330
150236,9153048000104,28726995034,0.3768
150237,92080878000100,83227113015,0.1050


In [115]:
graph = create_graph(arestas_1)

148517it [00:05, 26849.07it/s]


In [116]:
def get_concessions(list_nodes,emp_type): 
  # As concessões são todas emps marcadas como MANEJO,1
  # (fonte legal e extratores de madeira)

  count = 0
  for node in list_nodes:
    if emp_type[node] == 'MANEJO':
      count+=1

  return count

In [117]:

G_orig_weight = graph.copy()

In [118]:

def print_graph_metrics(graph, emp_type):
  # Print overall graph metrics

  g_aux = G_orig_weight.to_undirected()

  # Number of connected components in the graph
  components_len = []
  for item in nx.connected_components(g_aux):
    components_len.append((len(item),item))

  print(f'Total Graph: {graph}')
  print(f'Number of components: {len(components_len)}')
  print(f'Number of concessions: {get_concessions(graph.nodes(),emp_type)}')

  print()
  print()
  print('Components with more than 1000 nodes')
  for c in components_len:
    if c[0] > 1000:
      subg = graph.subgraph(c[1])

      print(f'Subgraph: {subg}')
      print(f'Number of concessions: {get_concessions(subg.nodes(),emp_type)}')
      print()
      print()


In [119]:
print_graph_metrics(G_orig_weight, emp_type)

Total Graph: DiGraph with 72094 nodes and 74552 edges
Number of components: 849
Number of concessions: 214


Components with more than 1000 nodes
Subgraph: DiGraph with 66172 nodes and 69472 edges
Number of concessions: 180




In [120]:
G_orig_weight.nodes()

NodeView((78952082000161, 9402361000139, 470994000100, 75218883000100, 5422729000170, 76803972000186, 7027349000166, 8408127000156, 3281325000188, 82191941000114, 1699020000165, 73304982000189, 4385294000259, 8849597000155, 82222290000182, 79109088000134, 24619519000110, 1473384000122, 85042737000120, 77540078000123, 20416094000191, 147816000142, 2164300000131, 24146864000184, 81120255000190, 3411505000137, 23672158000103, 4966423000111, 8113423000120, 82273939000194, 7716364000111, 78943925000163, 976181000196, 78442936000160, 81260044000152, 2411968000136, 9089341000150, 2181280000107, 7120780000152, 4803423000109, 85491520000105, 3407641000153, 26977992000103, 22488714000123, 6992711000176, 6238274000108, 20681182000110, 78445822000173, 167097000121, 7728113000157, 1985870000120, 75274423002738, 25134467000155, 4104090000111, 3767185000152, 23637106000104, 4590704000112, 82252099000183, 12985761000129, 629156000136, 13645548000130, 8664960000168, 2928483000114, 26147551000176, 36994

In [121]:
node_df['is_bridge_linkage'] =False
node_df

,CNPJ_CPF,type,source,is_bridge_linkage
0,78952082000161,PTO_IBAMA,Remetente,False
1,9402361000139,PTO_IBAMA,Remetente,False
3,470994000100,PTO_IBAMA,Remetente,False
4,75218883000100,PTO_IBAMA,Remetente,False
5,5422729000170,PTO_IBAMA,Remetente,False
...,...,...,...,...
300472,28638514034,FINAL,Destinatário,False
300473,58371249004,FINAL,Destinatário,False
300474,50911406034,FINAL,Destinatário,False
300476,83227113015,FINAL,Destinatário,False


In [122]:
node_df['degree']= 0
node_df

,CNPJ_CPF,type,source,is_bridge_linkage,degree
0,78952082000161,PTO_IBAMA,Remetente,False,0
1,9402361000139,PTO_IBAMA,Remetente,False,0
3,470994000100,PTO_IBAMA,Remetente,False,0
4,75218883000100,PTO_IBAMA,Remetente,False,0
5,5422729000170,PTO_IBAMA,Remetente,False,0
...,...,...,...,...,...
300472,28638514034,FINAL,Destinatário,False,0
300473,58371249004,FINAL,Destinatário,False,0
300474,50911406034,FINAL,Destinatário,False,0
300476,83227113015,FINAL,Destinatário,False,0


In [123]:
node_manejo = node_df[node_df['type']=="MANEJO"]
node_manejo

,CNPJ_CPF,type,source,is_bridge_linkage,degree
2587,36711381149,MANEJO,Remetente,False,0
2589,47445033815,MANEJO,Remetente,False,0
2590,254100163,MANEJO,Remetente,False,0
2598,63601769000185,MANEJO,Remetente,False,0
2599,65596838268,MANEJO,Remetente,False,0
...,...,...,...,...,...
147812,5494941973,MANEJO,Remetente,False,0
149962,61362573272,MANEJO,Remetente,False,0
149985,50979671272,MANEJO,Remetente,False,0
150009,9373192957,MANEJO,Remetente,False,0


#Walleria Way

In [124]:
df_tran = transporte_primeiro_semestre[['CPF_CNPJ_Rem', 'TpRem', 'CPF_CNPJ_Des', 'TpDes', 'Volume']]
df_tran = df_tran.groupby(['CPF_CNPJ_Rem', 'TpRem', 'CPF_CNPJ_Des', 'TpDes'])['Volume'].sum().reset_index()


In [125]:
df_tran

,CPF_CNPJ_Rem,TpRem,CPF_CNPJ_Des,TpDes,Volume
0,253169151,MANEJO,11602746000191,PTO_IBAMA,23.7704
1,254100163,MANEJO,4642220000170,PTO_IBAMA,27.5790
2,657233170,MANEJO,75558817000189,PTO_IBAMA,106.1130
3,964767708,MANEJO,16716007000144,PTO_IBAMA,70.0000
4,964767708,MANEJO,24090101000169,PTO_IBAMA,53.3850
...,...,...,...,...,...
74688,97520331000275,PTO_IBAMA,7933304249,FINAL,0.1680
74689,97520331000275,PTO_IBAMA,90658620215,FINAL,0.3150
74690,97529804000114,PTO_IBAMA,20982589000188,PTO_IBAMA,72.1580
74691,97538233000184,PTO_IBAMA,5219072927,FINAL,2.7500


In [126]:
df_tran['CPF_CNPJ_Rem'] = df_tran['CPF_CNPJ_Rem'].astype(str)
df_tran['CPF_CNPJ_Des'] = df_tran['CPF_CNPJ_Des'].astype(str)

In [127]:
nodes_new = set(df_tran['CPF_CNPJ_Rem']).union(set(df_tran['CPF_CNPJ_Des']))

In [128]:
nodes_new

{'2661442847',
 '7346522591',
 '10769768000188',
 '83063650706',
 '14489000855',
 '24240478987',
 '1181005809',
 '66325382820',
 '34927281892',
 '21642767875',
 '98080458804',
 '6954924970',
 '142371807',
 '37669073870',
 '32825959804',
 '30977886972',
 '57055416787',
 '30916259153',
 '84570970753',
 '7516887978',
 '17047740000186',
 '179023551',
 '27777530836',
 '84339683191',
 '55338232120',
 '4841979590',
 '27374625847',
 '10409996700',
 '13401266802',
 '3662290871',
 '5685116462',
 '6090616400',
 '29462967172',
 '22468713877',
 '42873083875',
 '75919125853',
 '11418992000198',
 '10246202637',
 '79251013772',
 '8164357820',
 '74570692834',
 '9199740664',
 '6426373931',
 '20449615804',
 '3183602849',
 '47296003120',
 '2320118853',
 '3222305862',
 '46775943415',
 '14554663876',
 '25136834000150',
 '57243727972',
 '38052369649',
 '3950396810',
 '15766672861',
 '40313166900',
 '21319680860',
 '20077718968',
 '89138490897',
 '4914110148',
 '40166945234',
 '828996814',
 '17275227886',
 '5

In [129]:
G = nx.DiGraph()
G.add_nodes_from(nodes_new)

In [130]:

# Cria as arestas com base nas transações e com peso = volume
edges = []

for row in df_tran.iterrows():
    # Ignora laços
    if str(row[1]['CPF_CNPJ_Rem']) != str(row[1]['CPF_CNPJ_Des']):
        edges.append((str(row[1]['CPF_CNPJ_Rem']), str(row[1]['CPF_CNPJ_Des']), {'Volume': row[1]['Volume']}))
G.add_edges_from(edges)

In [131]:
df_tran[df_tran['CPF_CNPJ_Des']=='356********']

,CPF_CNPJ_Rem,TpRem,CPF_CNPJ_Des,TpDes,Volume
44041,15721184000156,PTO_IBAMA,356********,PTO_IBAMA,2.789
44803,16904840000119,PTO_IBAMA,356********,PTO_IBAMA,3.048


In [132]:
df_pto = df_tran[(df_tran['TpRem'] == 'PTO_IBAMA') & (df_tran['TpDes'] == 'PTO_IBAMA')]
df_pto = df_pto.groupby('CPF_CNPJ_Rem')['Volume'].sum().reset_index()

In [133]:
df_pto

,CPF_CNPJ_Rem,Volume
0,1011441000151,20.0000
1,1011587000105,0.1130
2,10173601000150,8.5200
3,10194607000103,182.9440
4,10198258000106,29.5341
...,...,...
2411,9813168000190,0.5000
2412,981704000192,9.3027
2413,99007000102,17.9054
2414,9942474000127,7.2464


In [134]:
contagens = df_pto['CPF_CNPJ_Rem'].value_counts()
print(contagens[contagens > 1])  # 

Series([], Name: count, dtype: int64)


In [135]:
nodes_pto = set(df_pto['CPF_CNPJ_Rem'])
nodes_pto

{'34703017000127',
 '15302028000150',
 '10769768000188',
 '81868390000118',
 '2879345000273',
 '4375970000122',
 '34478149000100',
 '10419974000168',
 '18114596000116',
 '14428201000107',
 '19982686000191',
 '20335624000258',
 '7637195000124',
 '8563651000100',
 '19138008000147',
 '1064870000197',
 '10256272000100',
 '11140700000106',
 '95817664000154',
 '17921936000158',
 '1309298000189',
 '1962284000160',
 '8584159000102',
 '23680704000158',
 '27129817000110',
 '11236215000122',
 '14309392000180',
 '22842109000109',
 '21239917000113',
 '10248017000116',
 '72559586000210',
 '92572783000104',
 '955914000106',
 '11141020000107',
 '59784868000334',
 '24041994000152',
 '11587033000104',
 '16620673000184',
 '7018493000136',
 '22034016000158',
 '8742712000198',
 '21066743000134',
 '9813168000190',
 '5757158000124',
 '84290378000167',
 '27073299000160',
 '75558817000189',
 '1499475000137',
 '5992508000137',
 '9603593000155',
 '11418992000198',
 '17262412000100',
 '8491411000139',
 '172527380

In [136]:
emp_pto_degree = {}

for node in nodes_pto:
    emp_pto_degree[node]= G.degree(node)


emp_pto_degree

{'34703017000127': 4,
 '15302028000150': 15,
 '10769768000188': 5,
 '81868390000118': 34,
 '2879345000273': 6,
 '4375970000122': 1,
 '34478149000100': 1,
 '10419974000168': 25,
 '18114596000116': 2,
 '14428201000107': 11,
 '19982686000191': 4,
 '20335624000258': 6,
 '7637195000124': 14,
 '8563651000100': 11,
 '19138008000147': 2,
 '1064870000197': 4,
 '10256272000100': 53,
 '11140700000106': 44,
 '95817664000154': 141,
 '17921936000158': 21,
 '1309298000189': 2,
 '1962284000160': 6,
 '8584159000102': 23,
 '23680704000158': 6,
 '27129817000110': 27,
 '11236215000122': 1,
 '14309392000180': 2,
 '22842109000109': 25,
 '21239917000113': 175,
 '10248017000116': 10,
 '72559586000210': 15,
 '92572783000104': 2,
 '955914000106': 15,
 '11141020000107': 5,
 '59784868000334': 48,
 '24041994000152': 7,
 '11587033000104': 13,
 '16620673000184': 49,
 '7018493000136': 11,
 '22034016000158': 14,
 '8742712000198': 2,
 '21066743000134': 1,
 '9813168000190': 1,
 '5757158000124': 25,
 '84290378000167': 2,

In [137]:
graus = list(emp_pto_degree.values())
graus

[4,
 15,
 5,
 34,
 6,
 1,
 1,
 25,
 2,
 11,
 4,
 6,
 14,
 11,
 2,
 4,
 53,
 44,
 141,
 21,
 2,
 6,
 23,
 6,
 27,
 1,
 2,
 25,
 175,
 10,
 15,
 2,
 15,
 5,
 48,
 7,
 13,
 49,
 11,
 14,
 2,
 1,
 1,
 25,
 2,
 2,
 38,
 1,
 6,
 19,
 4,
 14,
 7,
 3,
 224,
 30,
 5,
 4,
 223,
 290,
 3,
 18,
 17,
 7,
 4,
 26,
 2,
 2,
 11,
 21,
 4,
 181,
 162,
 26,
 7,
 10,
 3,
 1,
 10,
 1,
 27,
 17,
 4,
 4,
 1,
 1,
 5,
 61,
 22,
 9,
 15,
 12,
 67,
 9,
 2,
 2,
 20,
 19,
 4,
 7,
 9,
 4,
 1,
 1,
 3,
 69,
 119,
 2,
 5,
 1,
 1,
 33,
 26,
 2,
 5,
 5,
 153,
 4,
 11,
 103,
 24,
 4,
 18,
 2,
 30,
 3,
 25,
 3,
 18,
 12,
 70,
 5,
 32,
 9,
 0,
 13,
 20,
 158,
 7,
 1,
 6,
 1,
 15,
 2,
 52,
 5,
 15,
 5,
 25,
 1,
 1,
 3,
 1,
 3,
 7,
 3,
 3,
 34,
 26,
 70,
 16,
 1,
 8,
 10,
 7,
 3,
 1,
 1,
 13,
 4,
 58,
 96,
 3,
 29,
 10,
 53,
 46,
 41,
 2,
 2,
 28,
 5,
 13,
 6,
 3,
 12,
 2,
 1,
 1,
 31,
 9,
 1,
 3,
 10,
 17,
 5,
 2,
 18,
 13,
 1,
 6,
 4,
 14,
 18,
 1,
 3,
 14,
 6,
 2,
 4,
 7,
 7,
 12,
 7,
 9,
 3,
 13,
 6,
 5,
 25,
 4,
 4,
 18

In [138]:
print(f"Grau:    {graus}")
q1 = np.quantile(graus, 0.25)  # Primeiro quartil (25%)
q2 = np.quantile(graus, 0.5)   # Segundo quartil (50%, ou mediana)
q3 = np.quantile(graus, 0.75)  # Terceiro quartil (75%)
interquartil = q3-q1
limite_superior = q3 + 1.5*interquartil
print(f"Q1 (25%): {q1}")
print(f"Q2 (50% - Mediana): {q2}")
print(f"Q3 (75%): {q3}")
print(f"Limite superior: {limite_superior}")
print(f"Máximo: {max(graus)}")

Grau:    [4, 15, 5, 34, 6, 1, 1, 25, 2, 11, 4, 6, 14, 11, 2, 4, 53, 44, 141, 21, 2, 6, 23, 6, 27, 1, 2, 25, 175, 10, 15, 2, 15, 5, 48, 7, 13, 49, 11, 14, 2, 1, 1, 25, 2, 2, 38, 1, 6, 19, 4, 14, 7, 3, 224, 30, 5, 4, 223, 290, 3, 18, 17, 7, 4, 26, 2, 2, 11, 21, 4, 181, 162, 26, 7, 10, 3, 1, 10, 1, 27, 17, 4, 4, 1, 1, 5, 61, 22, 9, 15, 12, 67, 9, 2, 2, 20, 19, 4, 7, 9, 4, 1, 1, 3, 69, 119, 2, 5, 1, 1, 33, 26, 2, 5, 5, 153, 4, 11, 103, 24, 4, 18, 2, 30, 3, 25, 3, 18, 12, 70, 5, 32, 9, 0, 13, 20, 158, 7, 1, 6, 1, 15, 2, 52, 5, 15, 5, 25, 1, 1, 3, 1, 3, 7, 3, 3, 34, 26, 70, 16, 1, 8, 10, 7, 3, 1, 1, 13, 4, 58, 96, 3, 29, 10, 53, 46, 41, 2, 2, 28, 5, 13, 6, 3, 12, 2, 1, 1, 31, 9, 1, 3, 10, 17, 5, 2, 18, 13, 1, 6, 4, 14, 18, 1, 3, 14, 6, 2, 4, 7, 7, 12, 7, 9, 3, 13, 6, 5, 25, 4, 4, 188, 5, 21, 3, 8, 5, 10, 9, 3, 5, 1, 14, 5, 17, 68, 38, 2, 247, 7, 85, 22, 2, 4, 13, 12, 24, 21, 2, 30, 2, 11, 14, 2, 13, 7, 3, 9, 2, 1, 5, 2, 1, 1, 62, 4, 22, 8, 3, 64, 62, 21, 6, 116, 2, 3, 18, 40, 7, 131, 4, 43, 

In [139]:
manejos_outliers = 0 

for emp in graus:
  if emp>= limite_superior:
    manejos_outliers+=1

print(f"Numero de Empresas Patio outliers (importantes quanto grau): {manejos_outliers} ")



Numero de Empresas Patio outliers (importantes quanto grau): 318 


In [140]:
print(nx.number_weakly_connected_components(G))

818


In [143]:
df_pto['Grau'] = df_pto['CPF_CNPJ_Rem'].apply(lambda x: G.degree(x))
df_pto

,CPF_CNPJ_Rem,Volume,Grau
0,1011441000151,20.0000,3
1,1011587000105,0.1130,4
2,10173601000150,8.5200,22
3,10194607000103,182.9440,8
4,10198258000106,29.5341,202
...,...,...,...
2411,9813168000190,0.5000,1
2412,981704000192,9.3027,3
2413,99007000102,17.9054,9
2414,9942474000127,7.2464,247


In [145]:
df_pto_outliers = df_pto[df_pto['Grau']>= limite_superior]
df_pto_outliers

,CPF_CNPJ_Rem,Volume,Grau
4,10198258000106,29.5341,202
8,10256272000100,639.8343,53
11,10310351000152,14.6111,87
17,10373419000142,5.9944,111
25,10435661000101,9.5043,84
...,...,...,...
2382,95808259000170,69.1650,87
2384,95817664000154,123.5790,141
2386,9583332000110,133.6660,57
2409,976181000196,14.8830,118


In [151]:
df_pto_outliers.loc[:, 'is_bridge_linkage'] = False

In [152]:
df_pto_outliers

,CPF_CNPJ_Rem,Volume,Grau,Conected_Components,is_bridge_linkage
4,10198258000106,29.5341,202,1013,False
8,10256272000100,639.8343,53,849,False
11,10310351000152,14.6111,87,900,False
17,10373419000142,5.9944,111,924,False
25,10435661000101,9.5043,84,900,False
...,...,...,...,...,...
2382,95808259000170,69.1650,87,898,False
2384,95817664000154,123.5790,141,954,False
2386,9583332000110,133.6660,57,839,False
2409,976181000196,14.8830,118,934,False


In [153]:
print("Número de components conexas:")
numero_original_de_componentes = nx.number_weakly_connected_components(G) 
print(nx.number_weakly_connected_components(G))

Número de components conexas:
818


In [ ]:
components = []
for node in list(df_pto_outliers['CPF_CNPJ_Rem']):
    SG = nx.subgraph_view(G, filter_node= lambda x: x != node)
    components.append(nx.number_weakly_connected_components(SG))



        CPF_CNPJ_Rem    Volume  Grau
0      1011441000151   20.0000     3
1      1011587000105    0.1130     4
2     10173601000150    8.5200    22
3     10194607000103  182.9440     8
4     10198258000106   29.5341   202
...              ...       ...   ...
2411   9813168000190    0.5000     1
2412    981704000192    9.3027     3
2413     99007000102   17.9054     9
2414   9942474000127    7.2464   247
2415   9943001000144    1.0660     9

[2416 rows x 3 columns]


C:\Users\jaime\AppData\Local\Temp\ipykernel_16824\1800842197.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pto_outliers['Conected_Components'] = components


In [157]:
components

[1013,
 849,
 900,
 924,
 900,
 899,
 871,
 947,
 894,
 1450,
 953,
 879,
 1013,
 867,
 1019,
 1054,
 867,
 893,
 979,
 1844,
 1230,
 1694,
 902,
 941,
 867,
 874,
 877,
 859,
 1527,
 885,
 941,
 914,
 883,
 948,
 985,
 1205,
 884,
 886,
 999,
 975,
 881,
 1191,
 1054,
 935,
 1260,
 872,
 929,
 935,
 889,
 854,
 923,
 887,
 874,
 863,
 927,
 892,
 1492,
 913,
 978,
 919,
 867,
 1090,
 928,
 905,
 875,
 1020,
 946,
 885,
 871,
 870,
 927,
 961,
 1008,
 884,
 824,
 877,
 988,
 970,
 897,
 910,
 883,
 1011,
 986,
 862,
 995,
 874,
 871,
 913,
 877,
 864,
 877,
 1108,
 862,
 889,
 1010,
 883,
 973,
 919,
 923,
 886,
 910,
 879,
 870,
 921,
 896,
 824,
 911,
 1086,
 963,
 821,
 976,
 901,
 931,
 880,
 936,
 887,
 872,
 914,
 1026,
 879,
 888,
 916,
 903,
 878,
 945,
 875,
 876,
 1110,
 899,
 929,
 875,
 955,
 980,
 949,
 878,
 908,
 913,
 910,
 996,
 903,
 880,
 873,
 873,
 895,
 1127,
 877,
 918,
 1305,
 876,
 895,
 875,
 868,
 988,
 1198,
 874,
 897,
 878,
 1438,
 975,
 898,
 874,
 907,
 

In [154]:
df_pto_outliers.loc[df_pto_outliers['Conected_Components'] > numero_original_de_componentes, 'is_bridge_linkage'] = True
df_pto_outliers

,CPF_CNPJ_Rem,Volume,Grau,Conected_Components,is_bridge_linkage
4,10198258000106,29.5341,202,1013,True
8,10256272000100,639.8343,53,849,True
11,10310351000152,14.6111,87,900,True
17,10373419000142,5.9944,111,924,True
25,10435661000101,9.5043,84,900,True
...,...,...,...,...,...
2382,95808259000170,69.1650,87,898,True
2384,95817664000154,123.5790,141,954,True
2386,9583332000110,133.6660,57,839,True
2409,976181000196,14.8830,118,934,True


In [155]:
qtd_de_pontes_de_articulacao =  df_pto_outliers['is_bridge_linkage'].sum()
print(f"Das empresas importantes, quantas são ponts de  de articulação: {qtd_de_pontes_de_articulacao}")

Das empresas importantes, quantas são ponts de  de articulação: 318


## Encontrando fontes e sumidouros

In [158]:
emp_type= {}
df_tran

,CPF_CNPJ_Rem,TpRem,CPF_CNPJ_Des,TpDes,Volume
0,253169151,MANEJO,11602746000191,PTO_IBAMA,23.7704
1,254100163,MANEJO,4642220000170,PTO_IBAMA,27.5790
2,657233170,MANEJO,75558817000189,PTO_IBAMA,106.1130
3,964767708,MANEJO,16716007000144,PTO_IBAMA,70.0000
4,964767708,MANEJO,24090101000169,PTO_IBAMA,53.3850
...,...,...,...,...,...
74688,97520331000275,PTO_IBAMA,7933304249,FINAL,0.1680
74689,97520331000275,PTO_IBAMA,90658620215,FINAL,0.3150
74690,97529804000114,PTO_IBAMA,20982589000188,PTO_IBAMA,72.1580
74691,97538233000184,PTO_IBAMA,5219072927,FINAL,2.7500


In [159]:
origem = transporte_primeiro_semestre[["CPF_CNPJ_Rem", "LatOrigem", "LongOrigem", "TpRem"]].rename(
    columns={
    'CPF_CNPJ_Rem': 'CPF_CNPJ', 
    'LatOrigem': 'Latitude', 
    'LongOrigem': 'Longitude', 
    'TpRem': 'Tipo'
})
destino = transporte_primeiro_semestre[["CPF_CNPJ_Des", "LatDestino", "LongDestino", "TpDes"]].rename(
    columns={
    'CPF_CNPJ_Des': 'CPF_CNPJ', 
    'LatDestino': 'Latitude', 
    'LongDestino': 'Longitude', 
    'TpDes': 'Tipo'
})

nodes = pd.concat([origem, destino], ignore_index=True)
nodes.drop_duplicates("CPF_CNPJ", inplace=True)
nodes["Coordenada"] = nodes.apply(lambda row: (row['Latitude'], row['Longitude']), axis=1)
nodes.drop(["Latitude", "Longitude"], axis=1, inplace=True)
nodes.to_csv("nodes.csv", index=False)

In [161]:
edges = transporte_primeiro_semestre[["CPF_CNPJ_Rem", "CPF_CNPJ_Des", "Produto", "Volume", "DtEmissao"]]
edges = edges[edges['CPF_CNPJ_Rem'] != edges['CPF_CNPJ_Des']]
# edges["DtEmissao"] = pd.to_datetime(edges["DtEmissao"], yearfirst=True)
edges.sort_values(['DtEmissao'], ascending=True, inplace=True)
edges.to_csv("edges.csv", index=False)

In [162]:
nodes

,CPF_CNPJ,Tipo,Coordenada
0,78952082000161,PTO_IBAMA,"(-25.291667, -49.224167)"
1,9402361000139,PTO_IBAMA,"(-25.427778, -49.273056)"
3,470994000100,PTO_IBAMA,"(-25.615833, -49.344222)"
4,75218883000100,PTO_IBAMA,"(-23.279972, -51.069444)"
5,5422729000170,PTO_IBAMA,"(-24.98075, -53.472722)"
...,...,...,...
300472,28638514034,FINAL,"(nan, nan)"
300473,58371249004,FINAL,"(nan, nan)"
300474,50911406034,FINAL,"(nan, nan)"
300476,83227113015,FINAL,"(nan, nan)"


In [163]:
edges

,CPF_CNPJ_Rem,CPF_CNPJ_Des,Produto,Volume,DtEmissao
19218,5583309000175,18996153000104,Madeira serrada (prancha),7.6230,01/07/2017
1564,2411968000136,45036047915,Madeira serrada (vigota),0.1575,01/07/2017
16041,5669339000107,26441400804,Madeira serrada (tábua),0.0276,01/07/2017
5465,5669339000107,80205062849,Madeira serrada (tábua),0.0069,01/07/2017
16222,55844229000366,3750275807,Madeira serrada (caibro),0.0204,01/07/2017
...,...,...,...,...,...
147539,10315322268,3957205000158,Tora,4.6030,31/12/2017
148621,10315322268,3957205000158,Tora,2.1020,31/12/2017
147834,27176479291,14743232000144,Tora,35.2216,31/12/2017
148348,2495933190,34740696000104,Tora,4.0210,31/12/2017


In [164]:
def get_concessions(list_nodes,emp_type):
  # Concessions are all emps which are marked as MANEJO,
  # (legal source and extractors of timber)

  count = 0
  for node in list_nodes:
    if emp_type[node] == 'MANEJO':
      count+=1

  return count

In [168]:
emps = pd.read_csv('../data/nodes.csv')

emp_type = {}

for i,node in emps.iterrows():
  emp_type[node['CPF_CNPJ']] = node['Tipo']
  
emp_type

{'78952082000161': 'PTO_IBAMA',
 '9402361000139': 'PTO_IBAMA',
 '470994000100': 'PTO_IBAMA',
 '75218883000100': 'PTO_IBAMA',
 '5422729000170': 'PTO_IBAMA',
 '76803972000186': 'PTO_IBAMA',
 '7027349000166': 'PTO_IBAMA',
 '8408127000156': 'PTO_IBAMA',
 '3281325000188': 'PTO_IBAMA',
 '82191941000114': 'PTO_IBAMA',
 '1699020000165': 'PTO_IBAMA',
 '73304982000189': 'PTO_IBAMA',
 '4385294000259': 'PTO_IBAMA',
 '8849597000155': 'PTO_IBAMA',
 '82222290000182': 'PTO_IBAMA',
 '79109088000134': 'PTO_IBAMA',
 '24619519000110': 'PTO_IBAMA',
 '1473384000122': 'PTO_IBAMA',
 '85042737000120': 'PTO_IBAMA',
 '77540078000123': 'PTO_IBAMA',
 '20416094000191': 'PTO_IBAMA',
 '147816000142': 'PTO_IBAMA',
 '2164300000131': 'PTO_IBAMA',
 '24146864000184': 'PTO_IBAMA',
 '81120255000190': 'PTO_IBAMA',
 '3411505000137': 'PTO_IBAMA',
 '23672158000103': 'PTO_IBAMA',
 '4966423000111': 'PTO_IBAMA',
 '8113423000120': 'PTO_IBAMA',
 '82273939000194': 'PTO_IBAMA',
 '7716364000111': 'PTO_IBAMA',
 '78943925000163': 'PTO_IB

In [169]:
def get_concessions(list_nodes,emp_type):
  # Concessions are all emps which are marked as MANEJO,
  # (legal source and extractors of timber)

  count = 0
  for node in list_nodes:
    if emp_type[node] == 'MANEJO':
      count+=1

  return count

In [170]:
def get_sink_nodes(graph, emp_type):
  # If node is marked as FINAL, he is a sink
  # It is the final destination of the timber chain

  nodes = {}
  for node in graph.nodes():
    if emp_type[node] == 'FINAL':
      nodes[node] = 1
      continue

    # we consider sink nodes as final nodes or nodes that only transports to other final nodes
    not_sink = False
    for edge in graph.edges(node):
      if emp_type[edge[1]] != 'FINAL':
        not_sink=True

    if not not_sink:
      nodes[node] = 1

  return nodes

In [176]:
def get_timberflow(graph,emp_type):
  sink_nodes = get_sink_nodes(graph, emp_type)

  total_in = 0
  total_out = 0

  # For all edges in the graph
  # 1. sum the volume of input (from MANEJO types)
  # 2. sum the volume of output (from FINAL types)
  for edge in graph.edges():
    if emp_type[edge[0]] == 'MANEJO':
      total_in += path_weight(graph, [edge[0],edge[1]], weight='Volume')
    elif edge[1] in sink_nodes:
      total_out += path_weight(graph, [edge[0],edge[1]], weight='Volume')

  print(f'Inflow Vol(m3): {total_in} \nOut Vol(m3): {total_out} \nProportion: {total_out/total_in}')

In [177]:
get_timberflow(G, emp_type)

Inflow Vol(m3): 48886.936299999965 
Out Vol(m3): 123529.3651999959 
Proportion: 2.5268379356387687


In [ ]:
# TODO
# Para  cada componente conexa, cria uma rede de fluxo se tive manejo e final
# Se não tiver um dos dois, não faça 

